In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    BBB_Train_Decile4 = pd.read_pickle("BBB_Train_Decile4.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile4.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile4, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 4')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 4s 1ms/step - loss: 0.4595 - acc: 0.7966 - val_loss: 0.4509 - val_acc: 0.8057
Epoch 39/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4546 - acc: 0.8053 - val_loss: 0.4518 - val_acc: 0.8057
Epoch 40/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4560 - acc: 0.8053 - val_loss: 0.4503 - val_acc: 0.8083
Epoch 41/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4560 - acc: 0.8079 - val_loss: 0.4499 - val_acc: 0.8109
Epoch 42/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4507 - acc: 0.8073 - val_loss: 0.4496 - val_acc: 0.8109
Epoch 43/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4547 - acc: 0.8034 - val_loss: 0.4482 - val_acc: 0.8083
Epoch 44/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4527 - acc: 0.8060 - val_loss: 0.4492 - val_acc: 0.8070
Epoch 45/100
3087/3087 [==============================] - 4s 

Epoch 98/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4357 - acc: 0.8124 - val_loss: 0.4403 - val_acc: 0.8187
Epoch 99/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4333 - acc: 0.8111 - val_loss: 0.4407 - val_acc: 0.8122
Epoch 100/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4387 - acc: 0.8095 - val_loss: 0.4403 - val_acc: 0.8161
Test accuracy: 0.8160621761658031
Train on 3087 samples, validate on 772 samples
Epoch 1/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.6182 - acc: 0.6505 - val_loss: 0.5642 - val_acc: 0.6878
Epoch 2/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5332 - acc: 0.7373 - val_loss: 0.5056 - val_acc: 0.7811
Epoch 3/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4941 - acc: 0.7904 - val_loss: 0.4762 - val_acc: 0.7915
Epoch 4/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4744 - acc: 0.8030 - val_l

3087/3087 [==============================] - 4s 1ms/step - loss: 0.4101 - acc: 0.8218 - val_loss: 0.4152 - val_acc: 0.8225
Epoch 57/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4067 - acc: 0.8260 - val_loss: 0.4154 - val_acc: 0.8225
Epoch 58/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4130 - acc: 0.8238 - val_loss: 0.4154 - val_acc: 0.8225
Epoch 59/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4071 - acc: 0.8231 - val_loss: 0.4156 - val_acc: 0.8225
Epoch 60/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4050 - acc: 0.8235 - val_loss: 0.4156 - val_acc: 0.8225
Epoch 61/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4043 - acc: 0.8280 - val_loss: 0.4158 - val_acc: 0.8225
Epoch 62/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4086 - acc: 0.8241 - val_loss: 0.4157 - val_acc: 0.8212
Epoch 63/150
3087/3087 [==============================] - 4s 

3087/3087 [==============================] - 4s 1ms/step - loss: 0.4037 - acc: 0.8273 - val_loss: 0.4163 - val_acc: 0.8225
Epoch 117/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4050 - acc: 0.8225 - val_loss: 0.4167 - val_acc: 0.8212
Epoch 118/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4051 - acc: 0.8283 - val_loss: 0.4166 - val_acc: 0.8212
Epoch 119/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4057 - acc: 0.8247 - val_loss: 0.4166 - val_acc: 0.8212
Epoch 120/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4049 - acc: 0.8231 - val_loss: 0.4166 - val_acc: 0.8212
Epoch 121/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4054 - acc: 0.8209 - val_loss: 0.4166 - val_acc: 0.8212
Epoch 122/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4072 - acc: 0.8238 - val_loss: 0.4166 - val_acc: 0.8199
Epoch 123/150
3087/3087 [==============================

...

Epoch 66/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4628 - acc: 0.8005 - val_loss: 0.4633 - val_acc: 0.8135
Epoch 67/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4600 - acc: 0.8034 - val_loss: 0.4629 - val_acc: 0.8135
Epoch 68/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4600 - acc: 0.8030 - val_loss: 0.4630 - val_acc: 0.8122
Epoch 69/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4613 - acc: 0.8056 - val_loss: 0.4629 - val_acc: 0.8122
Epoch 70/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4637 - acc: 0.8066 - val_loss: 0.4628 - val_acc: 0.8135
Epoch 71/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4631 - acc: 0.8021 - val_loss: 0.4631 - val_acc: 0.8122
Epoch 72/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4617 - acc: 0.8037 - val_loss: 0.4631 - val_acc: 0.8122
Epoch 73/100
3087/3087 [========================

3087/3087 [==============================] - 4s 1ms/step - loss: 0.4215 - acc: 0.8218 - val_loss: 0.4207 - val_acc: 0.8135
Epoch 26/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4249 - acc: 0.8231 - val_loss: 0.4197 - val_acc: 0.8135

Epoch 00026: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 27/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4268 - acc: 0.8196 - val_loss: 0.4196 - val_acc: 0.8135
Epoch 28/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4193 - acc: 0.8257 - val_loss: 0.4194 - val_acc: 0.8135
Epoch 29/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4234 - acc: 0.8241 - val_loss: 0.4195 - val_acc: 0.8122
Epoch 30/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4178 - acc: 0.8225 - val_loss: 0.4195 - val_acc: 0.8122
Epoch 31/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4226 - acc: 0.8199 - val_loss: 0.41

3087/3087 [==============================] - 4s 1ms/step - loss: 0.4159 - acc: 0.8212 - val_loss: 0.4207 - val_acc: 0.8096
Epoch 85/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4211 - acc: 0.8264 - val_loss: 0.4206 - val_acc: 0.8096
Epoch 86/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4188 - acc: 0.8209 - val_loss: 0.4207 - val_acc: 0.8083
Epoch 87/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4251 - acc: 0.8218 - val_loss: 0.4206 - val_acc: 0.8083
Epoch 88/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4116 - acc: 0.8235 - val_loss: 0.4207 - val_acc: 0.8083
Epoch 89/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4223 - acc: 0.8244 - val_loss: 0.4206 - val_acc: 0.8083
Epoch 90/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4179 - acc: 0.8238 - val_loss: 0.4206 - val_acc: 0.8083
Epoch 91/200
3087/3087 [==============================] - 4s 

3087/3087 [==============================] - 4s 1ms/step - loss: 0.4223 - acc: 0.8267 - val_loss: 0.4212 - val_acc: 0.8083
Epoch 145/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4187 - acc: 0.8225 - val_loss: 0.4213 - val_acc: 0.8083
Epoch 146/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4185 - acc: 0.8215 - val_loss: 0.4215 - val_acc: 0.8070
Epoch 147/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4228 - acc: 0.8218 - val_loss: 0.4216 - val_acc: 0.8070
Epoch 148/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4155 - acc: 0.8196 - val_loss: 0.4217 - val_acc: 0.8070
Epoch 149/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4188 - acc: 0.8231 - val_loss: 0.4214 - val_acc: 0.8083
Epoch 150/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4178 - acc: 0.8199 - val_loss: 0.4214 - val_acc: 0.8083
Epoch 151/200
3087/3087 [==============================

3087/3087 [==============================] - 4s 1ms/step - loss: 0.5626 - acc: 0.7136 - val_loss: 0.5258 - val_acc: 0.7189
Epoch 4/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5260 - acc: 0.7318 - val_loss: 0.5054 - val_acc: 0.7396
Epoch 5/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5164 - acc: 0.7564 - val_loss: 0.5159 - val_acc: 0.7591
Epoch 6/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5114 - acc: 0.7664 - val_loss: 0.4944 - val_acc: 0.7889
Epoch 7/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4939 - acc: 0.7797 - val_loss: 0.4864 - val_acc: 0.7785
Epoch 8/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4848 - acc: 0.7878 - val_loss: 0.4757 - val_acc: 0.7772
Epoch 9/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4689 - acc: 0.7920 - val_loss: 0.4716 - val_acc: 0.7837
Epoch 10/50
3087/3087 [==============================] - 4s 1ms/step - lo

In [ ]:
0.8297